In [16]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("johnsmith88/heart-disease-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/heart-disease-dataset


In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

In [18]:
df = pd.read_csv(path + "/heart.csv")
df.shape

(1025, 14)

In [19]:
df.isna().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [25]:
X_train,X_test,y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

In [26]:
X_train

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
835,49,1,2,118,149,0,0,126,0,0.8,2,3,2
137,64,0,0,180,325,0,1,154,1,0.0,2,0,2
534,54,0,2,108,267,0,0,167,0,0.0,2,0,2
495,59,1,0,135,234,0,1,161,0,0.5,1,0,3
244,51,1,2,125,245,1,0,166,0,2.4,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,41,1,2,130,214,0,0,168,0,2.0,1,0,2
71,61,1,0,140,207,0,0,138,1,1.9,2,1,3
106,51,1,0,140,299,0,1,173,1,1.6,2,0,3
270,43,1,0,110,211,0,1,161,0,0.0,2,0,3


In [27]:
y_train.shape

(820,)

In [28]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
pre = logreg.predict(X_test)

In [31]:
accuracy_score(y_test, pre)

0.7804878048780488

In [34]:
d_tree = DecisionTreeClassifier()
d_tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [35]:
d_pred = d_tree.predict(X_test)
accuracy_score(y_test, d_pred)

0.9853658536585366

In [37]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

<h2>Normalization </h2>

In [38]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'trestbps', 'chol', 'thalach', 'oldpeak','cp','fbs','restecg','exang','oldpeak','slope','ca','thal']),
    ]
)

In [78]:
logreg_pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)


In [80]:
logreg_pipeline.fit(X_train, y_train)
pipe_pred =logreg_pipeline.predict(X_test)
accuracy_score(y_test, pipe_pred)

0.7853658536585366

In [81]:
decision_pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", DecisionTreeClassifier())]
)

In [82]:
decision_pipeline.fit(X_train, y_train)
pipe_pred =decision_pipeline.predict(X_test)
accuracy_score(y_test, pipe_pred)

0.9853658536585366

<h1>PCA</h1>

In [75]:
pca_pipeline = Pipeline(
    steps=[("preprocessor", preprocessor),('PCA',PCA(n_components=3)), ("classifier", DecisionTreeClassifier())]
)

In [77]:
pca_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalach', 'oldpeak', 'cp',
                                                   'fbs', 'restecg', 'exang',
                                                   'oldpeak', 'slope', 'ca',
                                                   'thal'])])),
                ('PCA', PCA(n_components=3)),
                ('classifier', DecisionTreeClassifier())])

In [83]:
pca_pipeline.fit(X_train, y_train)
pipe_pred =pca_pipeline.predict(X_test)
accuracy_score(y_test, pipe_pred)

0.9853658536585366